In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import time

# OpenWeather API Key
api_key = "d9cc2b43f4569d1bd791efaeac304dc7"
units = "metric"

In [2]:
# Import cities file as DataFrame
cities_pd = pd.read_csv("worldcities.csv")
cities_pd.head()

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
0,Qal eh-ye Now,Qal eh-ye,34.983000,63.133300,2997.0,Afghanistan,AF,AFG,Badghis
1,Chaghcharan,Chaghcharan,34.516701,65.250001,15000.0,Afghanistan,AF,AFG,Ghor
2,Lashkar Gah,Lashkar Gah,31.582998,64.360000,201546.0,Afghanistan,AF,AFG,Hilmand
3,Zaranj,Zaranj,31.112001,61.886998,49851.0,Afghanistan,AF,AFG,Nimroz
4,Tarin Kowt,Tarin Kowt,32.633298,65.866699,10000.0,Afghanistan,AF,AFG,Uruzgan


In [3]:
# Add columns for Temperature, Humidity, Cloudiness, Wind Speed
# Note that we used "" to specify initial entry.
cities_pd["Temperature_C"] = ""
cities_pd["Temperature_F"] = ""
cities_pd["Humidity"] = ""
cities_pd["Cloudiness"] = ""
cities_pd["Wind_Speed"] = ""

cities_pd.head()

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province,Temperature_C,Temperature_F,Humidity,Cloudiness,Wind_Speed
0,Qal eh-ye Now,Qal eh-ye,34.983000,63.133300,2997.0,Afghanistan,AF,AFG,Badghis,,,,,
1,Chaghcharan,Chaghcharan,34.516701,65.250001,15000.0,Afghanistan,AF,AFG,Ghor,,,,,
2,Lashkar Gah,Lashkar Gah,31.582998,64.360000,201546.0,Afghanistan,AF,AFG,Hilmand,,,,,
3,Zaranj,Zaranj,31.112001,61.886998,49851.0,Afghanistan,AF,AFG,Nimroz,,,,,
4,Tarin Kowt,Tarin Kowt,32.633298,65.866699,10000.0,Afghanistan,AF,AFG,Uruzgan,,,,,


In [8]:
# Random selection of 500 cities with a population >1000
# Using 50 cities to stay within OpenWeatherMap limit
selected_cities = cities_pd.sample(n=10)
selected_cities = selected_cities[selected_cities["pop"].astype(int) > 1000]

# View selected_cities
selected_cities.head()

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province,Temperature_C,Temperature_F,Humidity,Cloudiness,Wind_Speed
6211,Kovel,Kovel,51.217066,24.716620,68850.5,Ukraine,UA,UKR,Volyn,,,,,
6117,Gaziantep,Gaziantep,37.074984,37.384994,943262.0,Turkey,TR,TUR,Gaziantep,,,,,
5683,Matara,Matara,5.948977,80.542773,68244.0,Sri Lanka,LK,LKA,Matara,,,,,
1915,Jiaozuo,Jiaozuo,35.250005,113.220004,687270.0,China,CN,CHN,Henan,,,,,
7220,Vung Tau,Vung Tau,10.355374,107.084978,229225.0,Vietnam,VN,VNM,Bà Rịa–Vũng Tàu,,,,,
6549,Bullhead City,Bullhead City,35.148176,-114.567488,37989.0,United States of America,US,USA,Arizona,,,,,
6317,Centralia,Centralia,46.716411,-122.952971,16993.5,United States of America,US,USA,Washington,,,,,
6334,Tonopah,Tonopah,38.066990,-117.228979,1993.0,United States of America,US,USA,Nevada,,,,,
6418,Dalton,Dalton,34.769724,-84.970302,45077.5,United States of America,US,USA,Georgia,,,,,
6338,Farmington,Farmington,36.754151,-108.186094,42917.5,United States of America,US,USA,New Mexico,,,,,


In [6]:
# Loop through the cities_pd and run a temp search for each city
for index, row in cities_pd.iterrows():
    target_url = "http://api.openweathermap.org/data/2.5/weather?" 
        
    
    # Build query URL
    query_url = target_url + "appid=" + api_key + "&q=" + (row["city"])
    cities_data = requests.get(query_url).json()
    try: 
        selected_cities.set_value(index, "Temperature_C", cities_data["list"][0]["main"]["temp"])
        selected_cities.set_value(index, "Humidity", cities_data["list"][0]["main"]["humidity"])
        selected_cities.set_value(index, "Cloudiness", cities_data["list"][0]["main"]["clouds"])
        selected_cities.set_value(index, "Wind_Speed", cities_data["list"][0]["main"]["wind"]["speed"])
        print(target_url)    
                                                           
    except:
        #print("Error with data. Skipping")
        continue
                
# View selected_cities for added data
selected_cities.head()

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province,Temperature_C,Temperature_F,Humidity,Cloudiness,Wind_Speed
5046,Dombarovskiy,Dombarovskiy,50.754588,59.540024,5564.0,Russia,RU,RUS,Orenburg,,,,,
6022,Siliana,Siliana,36.083304,9.383302,26960.0,Tunisia,TN,TUN,Siliana,,,,,
5466,Bo,Bo,7.970016,-11.740018,170690.5,Sierra Leone,SL,SLE,Southern,,,,,
423,Kempsey,Kempsey,-31.087367,152.822031,10681.0,Australia,AU,AUS,New South Wales,,,,,
6166,Entebbe,Entebbe,0.060396,32.460024,127414.0,Uganda,UG,UGA,Wakiso,,,,,


In [ ]:
# Convert Celcius to Fahrenheit
        cel=int("Temperature_C")
        far=(9/5*(cel))+32
        selected_cities.set_value(index, "Temperature_F", far)
selected_cities.head()

In [ ]:
# Build a scatter plot for Temperature and Latitude
plt.scatter(selected_cities["Temperature_F"], 
            selected_cities["Latitude"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="City")

# Incorporate the other graph properties
plt.title("Temperature vs. Latitude by City")
plt.ylabel("Temperature_F")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([-2.5, 150])
plt.ylim([-2.5, 110000])

# Save the figure
plt.savefig("output_analysis/Temperature_Latitude.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for Humidity and Latitude
plt.scatter(selected_cities["Humidity"], 
            selected_cities["Latitude"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="City")

# Incorporate the other graph properties
plt.title("Humidity vs. Latitude by City")
plt.ylabel("Humidity")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([-2.5, 150])
plt.ylim([-2.5, 110000])

# Save the figure
plt.savefig("output_analysis/Humidity_Latitude.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for Cloudiness and Latitude
plt.scatter(selected_cities["Cloudiness"], 
            selected_cities["Latitude"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="City")

# Incorporate the other graph properties
plt.title("Cloudiness vs. Latitude by City")
plt.ylabel("Cloudiness")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([-2.5, 150])
plt.ylim([-2.5, 110000])

# Save the figure
plt.savefig("output_analysis/Cloudiness_Latitude.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for Wind_Speed and Latitude
plt.scatter(selected_cities["Wind_Speed"], 
            selected_cities["Latitude"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="City")

# Incorporate the other graph properties
plt.title("Wind_Speed vs. Latitude by City")
plt.ylabel("Wind_Speed")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([-2.5, 150])
plt.ylim([-2.5, 110000])

# Save the figure
plt.savefig("output_analysis/Wind_Speed_Latitude.png")

# Show plot
plt.show()